In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso
import copy

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.데이터 불러오기
1. '연구자 선정'
2. '상관관계 5'
3. '상관관계10'
4. '전체변수'
5. '(연구자선정).동일년도 외부변수, 과거예산포함'
6.  '(연구자선정).동일연도 외부변수, 과거예산 불포함 '
7.  '(상관관계).동일년도 외부변수, 과거예산포함'
8.  '(상관관계).동일연도 외부변수, 과거예산 불포함'
9. '33년 ARIMA'

In [14]:
#1. 파일 생성
writer=pd.ExcelWriter('장기추계_순차학습(아리마).xlsx', engine='openpyxl')

In [15]:
file_name_list =['과학기술 아리마', '과학기술 샤프_장기추계_ori', '고등교육 아리마', '고등교육 샤프_장기추계_ori','기후대기 아리마', '기후대기 샤프_장기추계_ori','식품 아리마', '식품 샤프_장기추계_ori']
file_name = file_name_list[6]
print(file_name)
excel_sheets = pd.read_excel(f'/content/drive/MyDrive/PC - Desktop/재정전망_동국대/최종데이터분석/1.데이터/2차데이터/{file_name}.xlsx')

식품 아리마


In [16]:
sheets = list(excel_sheets.keys())
file = excel_sheets
# file.drop('Unnamed: 0', axis = 1, inplace=True)
file.head()

,Unnamed: 0,식품안전예산,조직변수,공공질서및안전(기재부),합계출산율,소비자 물가지수,생산자 물가지수,식료품 및 비주류음료품(가계목적별 최종 소비지출)(실질)(원계열),주류 및 담배(가계목적별 최종 소비지출)(실질)(원계열),의료보건(가계목적별 최종 소비지출)(실질)(원계열),...,영양교육 및 상담 경험률(10~18세),가공식품 선택 시 영양표시 이용률(6~9세),가공식품 선택 시 영양표시 이용률(10~18세),영양섭취부족자 분율(1~9세),영양섭취부족자 분율(10~18세),에너지/지방과잉섭취자 분율(1~9세),에너지/지방과잉섭취자 분율(10~18세),건강식생활실천율(6~9세),건강식생활실천율(10~18세),청소년 비만율(중고등학생)
0,2009,44745,0,11001139,1.259,78.010,88.09,786787,187617,223357,...,19.0,7.6,27.4,13.6,18.3,1.5,2.0,33.7,37.2,5.3
1,2010,17223,0,11736095,1.192,81.656,95.63,813351,192255,240724,...,24.5,6.8,20.7,6.9,17.3,2.0,2.8,30.4,32.3,5.3
2,2011,28650,0,12490232,1.149,83.906,95.42,793787,183939,260735,...,25.1,9.0,24.4,8.4,18.9,2.4,2.6,34.0,36.3,5.1
3,2012,32906,0,13047826,1.226,86.373,99.06,814463,176996,277285,...,15.8,8.3,29.3,6.9,13.8,3.2,8.6,30.3,36.8,5.3
4,2013,40174,0,13796629,1.244,89.850,105.71,836140,176432,292973,...,17.2,7.4,27.8,5.8,13.2,5.4,5.4,36.3,37.1,5.6


## 2. 분석 시작


*   데이터셋을 window2로 만들어주는 def
*   모델들



In [17]:
final_models = []
real_values = []
pred_values = []

In [18]:
# 정규화 적용
def scaler(data):
    first_two_columns = data.iloc[:, :2]
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data.iloc[:, 2:])
    scaled_data_df = pd.DataFrame(scaled_data, columns=data.columns[2:])
    result = pd.concat([first_two_columns, scaled_data_df], axis=1)
    result.columns = data.columns
    # print(result)
    return result

scaler(file)

,Unnamed: 0,식품안전예산,조직변수,공공질서및안전(기재부),합계출산율,소비자 물가지수,생산자 물가지수,식료품 및 비주류음료품(가계목적별 최종 소비지출)(실질)(원계열),주류 및 담배(가계목적별 최종 소비지출)(실질)(원계열),의료보건(가계목적별 최종 소비지출)(실질)(원계열),...,영양교육 및 상담 경험률(10~18세),가공식품 선택 시 영양표시 이용률(6~9세),가공식품 선택 시 영양표시 이용률(10~18세),영양섭취부족자 분율(1~9세),영양섭취부족자 분율(10~18세),에너지/지방과잉섭취자 분율(1~9세),에너지/지방과잉섭취자 분율(10~18세),건강식생활실천율(6~9세),건강식생활실천율(10~18세),청소년 비만율(중고등학생)
0,2009,44745,0.0,0.000000,0.953202,0.000000,0.000000,0.000000,0.922147,0.000000,...,0.115942,0.519231,0.786517,1.000000,0.435897,0.000000,0.000000,0.151111,0.392,0.013245
1,2010,17223,0.0,0.037605,0.870690,0.080666,0.173853,0.085894,1.000000,0.037594,...,0.315217,0.365385,0.033708,0.396396,0.350427,0.059524,0.114286,0.004444,0.000,0.013245
2,2011,28650,0.0,0.076192,0.817734,0.130445,0.169011,0.022634,0.860409,0.080912,...,0.336957,0.788462,0.449438,0.531532,0.487179,0.107143,0.085714,0.164444,0.320,0.000000
3,2012,32906,0.0,0.104722,0.912562,0.185026,0.252940,0.089489,0.743865,0.116737,...,0.000000,0.653846,1.000000,0.396396,0.051282,0.202381,0.942857,0.000000,0.360,0.013245
4,2013,40174,0.0,0.143036,0.934729,0.261953,0.406272,0.159581,0.734398,0.150697,...,0.050725,0.480769,0.831461,0.297297,0.000000,0.464286,0.485714,0.266667,0.384,0.033113
5,2014,28284,0.0,0.187172,1.000000,0.305427,0.423104,0.160567,0.703109,0.189371,...,0.188406,0.711538,0.269663,0.441441,0.205128,0.357143,0.828571,0.071111,0.224,0.072848
6,2015,33123,1.0,0.220497,0.864532,0.331866,0.383906,0.178733,0.684879,0.231055,...,0.173913,0.173077,0.640449,0.234234,0.025641,0.607143,0.857143,0.551111,0.672,0.099338
7,2016,34135,1.0,0.253946,0.886700,0.358105,0.370994,0.236754,0.719811,0.272773,...,0.318841,0.750000,0.370787,0.288288,0.008547,0.619048,0.857143,0.386667,0.648,0.119205
8,2017,43707,1.0,0.322245,0.928571,0.372818,0.274614,0.270987,0.303455,0.309652,...,0.362319,0.615385,0.191011,0.288288,0.111111,0.416667,1.000000,0.546667,0.624,0.158940
9,2018,45897,1.0,0.349975,0.846059,0.393217,0.232649,0.319592,0.528502,0.360802,...,0.456522,0.192308,0.000000,0.324324,0.282051,0.392857,0.657143,0.720000,0.696,0.264901


In [19]:
def pred_2123(X, y, num_splits):
    train_test_splits = []
    for i in range(num_splits):
        X_train = X.iloc[:11 + i, :]
        y_train = y[1:12 + i]
        X_test = X.iloc[11 + i:12 + i, :]
        y_test = y[12 + i:13 + i]
        train_test_splits.append((X_train, y_train, X_test, y_test))
    print(train_test_splits)
    return train_test_splits

def pred_2021a(X, y, num_splits):
    train_test_splits = []
    for i in range(num_splits):
        X_train = X.iloc[:11 + i, :]
        y_train = y[1:12 + i]
        X_test = X.iloc[11 + i:12 + i, :]
        y_test = y[12 + i:13 + i]
        train_test_splits.append((X_train, y_train, X_test, y_test))
    print(train_test_splits)
    return train_test_splits

def pred_2021b(X, y, num_splits):
    train_test_splits = []
    for i in range(num_splits):
        X_train = X.iloc[:12 + i, :]
        y_train = y[1:13 + i]
        X_test = X.iloc[12 + i:13 + i, :]
        y_test = y[13 + i:14 + i]
        train_test_splits.append((X_train, y_train, X_test, y_test))
    print(train_test_splits)
    return train_test_splits

def pred_23n(X, y, num_splits):
    train_test_splits = []
    for i in range(num_splits):
        X_train = X.iloc[:11 + i, :]
        y_train = y[1:12 + i]
        X_test = X.iloc[11 + i:12 + i, :]
        y_test = y[12 + i:13 + i]
        train_test_splits.append((X_train, y_train, X_test, y_test))
    print(train_test_splits)
    return train_test_splits

def make_window_size2(data):

    raw = []
    days_in = 2

    for i in range(days_in, 0, -1):
        raw.append(data.shift(i-1))

    prep_inde = pd.concat(raw, axis=1)
    prep_inde.dropna(inplace=True)
    prep_inde.reset_index(drop=True, inplace=True)

    return prep_inde

win2_set = make_window_size2(file.iloc[:,2:].astype(float))

def pred_2123_multiple(dataset):
    return pred_2123(dataset, file.iloc[:, 1].values, 3)

def pred_2021_multiple_a(dataset):
    return pred_2021a(dataset, file.iloc[:, 1].values, 2)

def pred_2021_multiple_b(dataset):
    return pred_2021b(dataset, file.iloc[:, 1].values, 2)

def pred_23n_multiple(dataset):
    return pred_23n(dataset, file.iloc[:, 1].values, 13)

In [20]:
# window_size = 2 데이터세트 준비
scaler_data = scaler(file)
win2_set = make_window_size2(scaler_data.iloc[:,2:].astype(float))

train_test_splits = pred_23n_multiple(win2_set)

[(    조직변수  공공질서및안전(기재부)     합계출산율  소비자 물가지수  생산자 물가지수  \
0    0.0      0.000000  0.953202  0.000000  0.000000   
1    0.0      0.037605  0.870690  0.080666  0.173853   
2    0.0      0.076192  0.817734  0.130445  0.169011   
3    0.0      0.104722  0.912562  0.185026  0.252940   
4    0.0      0.143036  0.934729  0.261953  0.406272   
5    0.0      0.187172  1.000000  0.305427  0.423104   
6    1.0      0.220497  0.864532  0.331866  0.383906   
7    1.0      0.253946  0.886700  0.358105  0.370994   
8    1.0      0.322245  0.928571  0.372818  0.274614   
9    1.0      0.349975  0.846059  0.393217  0.232649   
10   1.0      0.385086  0.698276  0.434412  0.310814   

    식료품 및 비주류음료품(가계목적별 최종 소비지출)(실질)(원계열)  주류 및 담배(가계목적별 최종 소비지출)(실질)(원계열)  \
0                               0.000000                         0.922147   
1                               0.085894                         1.000000   
2                               0.022634                         0.860409   
3                

In [21]:
# 1.Lasso
for X_train, y_train, X_test, y_test in train_test_splits:
    lasso = Lasso()
    param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 3, 5, 10, 100]}
    grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    print("최적의 하이퍼파라미터:", grid_search.best_params_)
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(X_test)
    print('================x_test')
    print(X_test)
    final_models.append(best_model)
    real_values.append(y_test)
    pred_values.append(predictions)

print(final_models)
print(real_values)
print(pred_values)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.817e+04, tolerance: 5.108e+04
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.063e+05, tolerance: 1.020e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.088e+05, tolerance: 1.081e

최적의 하이퍼파라미터: {'alpha': 1}
================x_test
    조직변수  공공질서및안전(기재부)     합계출산율  소비자 물가지수  생산자 물가지수  \
11   1.0      0.431567  0.605911  0.466294  0.354854   

    식료품 및 비주류음료품(가계목적별 최종 소비지출)(실질)(원계열)  주류 및 담배(가계목적별 최종 소비지출)(실질)(원계열)  \
11                              0.379744                         0.478867   

    의료보건(가계목적별 최종 소비지출)(실질)(원계열)  국내총생산(실질)  기대여명(0세)(전체)  ...  \
11                      0.444474   0.475398      0.454545  ...   

    영양교육 및 상담 경험률(10~18세)  가공식품 선택 시 영양표시 이용률(6~9세)  \
11               0.807971                  0.480769   

    가공식품 선택 시 영양표시 이용률(10~18세)  영양섭취부족자 분율(1~9세)  영양섭취부족자 분율(10~18세)  \
11                     0.41573          0.189189            0.521368   

    에너지/지방과잉섭취자 분율(1~9세)  에너지/지방과잉섭취자 분율(10~18세)  건강식생활실천율(6~9세)  \
11              0.595238                0.857143        0.355556   

    건강식생활실천율(10~18세)  청소년 비만율(중고등학생)  
11             0.504        0.397351  

[1 rows x 106 columns]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.877e+04, tolerance: 6.086e+04
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.348e+05, tolerance: 1.252e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.449e+05, tolerance: 1.111e

최적의 하이퍼파라미터: {'alpha': 5}
================x_test
    조직변수  공공질서및안전(기재부)     합계출산율  소비자 물가지수  생산자 물가지수  \
12   1.0      0.487003  0.533251  0.474701  0.355315   

    식료품 및 비주류음료품(가계목적별 최종 소비지출)(실질)(원계열)  주류 및 담배(가계목적별 최종 소비지출)(실질)(원계열)  \
12                              0.452206                         0.442106   

    의료보건(가계목적별 최종 소비지출)(실질)(원계열)  국내총생산(실질)  기대여명(0세)(전체)  ...  \
12                      0.525318   0.512179      0.532468  ...   

    영양교육 및 상담 경험률(10~18세)  가공식품 선택 시 영양표시 이용률(6~9세)  \
12               0.605072                       0.0   

    가공식품 선택 시 영양표시 이용률(10~18세)  영양섭취부족자 분율(1~9세)  영양섭취부족자 분율(10~18세)  \
12                    0.460674          0.369369                 1.0   

    에너지/지방과잉섭취자 분율(1~9세)  에너지/지방과잉섭취자 분율(10~18세)  건강식생활실천율(6~9세)  \
12              0.666667                0.614286        0.644444   

    건강식생활실천율(10~18세)  청소년 비만율(중고등학생)  
12              0.52        0.463576  

[1 rows x 106 columns]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.139e+05, tolerance: 8.598e+04
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.595e+05, tolerance: 1.385e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.414e+05, tolerance: 1.107e

...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.144e+05, tolerance: 3.867e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.201e+06, tolerance: 2.945e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.176e+06, tolerance: 5.445e

최적의 하이퍼파라미터: {'alpha': 3}
================x_test
    조직변수  공공질서및안전(기재부)     합계출산율  소비자 물가지수  생산자 물가지수  \
22   1.0      0.916525  0.080049  0.914135   0.91146   

    식료품 및 비주류음료품(가계목적별 최종 소비지출)(실질)(원계열)  주류 및 담배(가계목적별 최종 소비지출)(실질)(원계열)  \
22                              0.914915                         0.075536   

    의료보건(가계목적별 최종 소비지출)(실질)(원계열)  국내총생산(실질)  기대여명(0세)(전체)  ...  \
22                      0.916452   0.916358      0.909091  ...   

    영양교육 및 상담 경험률(10~18세)  가공식품 선택 시 영양표시 이용률(6~9세)  \
22               0.960145                  0.173077   

    가공식품 선택 시 영양표시 이용률(10~18세)  영양섭취부족자 분율(1~9세)  영양섭취부족자 분율(10~18세)  \
22                    0.460674          0.036036            0.717949   

    에너지/지방과잉섭취자 분율(1~9세)  에너지/지방과잉섭취자 분율(10~18세)  건강식생활실천율(6~9세)  \
22              0.964286                0.871429        0.964444   

    건강식생활실천율(10~18세)  청소년 비만율(중고등학생)  
22             0.968        0.960265  

[1 rows x 106 columns]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.666e+05, tolerance: 4.400e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.666e+06, tolerance: 3.398e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.294e+06, tolerance: 6.049e

최적의 하이퍼파라미터: {'alpha': 3}
================x_test
    조직변수  공공질서및안전(기재부)    합계출산율  소비자 물가지수  생산자 물가지수  \
23   1.0      0.958219  0.04064  0.957057   0.95573   

    식료품 및 비주류음료품(가계목적별 최종 소비지출)(실질)(원계열)  주류 및 담배(가계목적별 최종 소비지출)(실질)(원계열)  \
23                               0.95738                         0.037768   

    의료보건(가계목적별 최종 소비지출)(실질)(원계열)  국내총생산(실질)  기대여명(0세)(전체)  ...  \
23                      0.958224   0.958179      0.961039  ...   

    영양교육 및 상담 경험률(10~18세)  가공식품 선택 시 영양표시 이용률(6~9세)  \
23                    1.0                  0.173077   

    가공식품 선택 시 영양표시 이용률(10~18세)  영양섭취부족자 분율(1~9세)  영양섭취부족자 분율(10~18세)  \
23                    0.449438               0.0             0.74359   

    에너지/지방과잉섭취자 분율(1~9세)  에너지/지방과잉섭취자 분율(10~18세)  건강식생활실천율(6~9세)  \
23                   1.0                0.914286             1.0   

    건강식생활실천율(10~18세)  청소년 비만율(중고등학생)  
23               1.0             1.0  

[1 rows x 106 columns]
[Lasso(alpha=1), Lasso(alpha=5), Lasso(alpha=100), Lasso(

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.617e+05, tolerance: 6.133e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.922e+05, tolerance: 4.400e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.801e+06, tolerance: 7.108e

In [22]:
# 2. Ridge 모델
for X_train, y_train, X_test, y_test in train_test_splits:
    ridge = Ridge()
    param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 3, 5, 10, 100]}
    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    print("최적의 하이퍼파라미터:", grid_search.best_params_)
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(X_test)
    print('================x_test')
    print(X_test)
    final_models.append(best_model)
    real_values.append(y_test)
    pred_values.append(predictions)

print(final_models)
print(real_values)
print(pred_values)

최적의 하이퍼파라미터: {'alpha': 1}
================x_test
    조직변수  공공질서및안전(기재부)     합계출산율  소비자 물가지수  생산자 물가지수  \
11   1.0      0.431567  0.605911  0.466294  0.354854   

    식료품 및 비주류음료품(가계목적별 최종 소비지출)(실질)(원계열)  주류 및 담배(가계목적별 최종 소비지출)(실질)(원계열)  \
11                              0.379744                         0.478867   

    의료보건(가계목적별 최종 소비지출)(실질)(원계열)  국내총생산(실질)  기대여명(0세)(전체)  ...  \
11                      0.444474   0.475398      0.454545  ...   

    영양교육 및 상담 경험률(10~18세)  가공식품 선택 시 영양표시 이용률(6~9세)  \
11               0.807971                  0.480769   

    가공식품 선택 시 영양표시 이용률(10~18세)  영양섭취부족자 분율(1~9세)  영양섭취부족자 분율(10~18세)  \
11                     0.41573          0.189189            0.521368   

    에너지/지방과잉섭취자 분율(1~9세)  에너지/지방과잉섭취자 분율(10~18세)  건강식생활실천율(6~9세)  \
11              0.595238                0.857143        0.355556   

    건강식생활실천율(10~18세)  청소년 비만율(중고등학생)  
11             0.504        0.397351  

[1 rows x 106 columns]
최적의 하이퍼파라미터: {'alpha': 1}
================x_test
    조직

In [23]:
# 3. Linear 모델
for X_train, y_train, X_test, y_test in train_test_splits:
    linear = LinearRegression()
    linear.fit(X_train, y_train)
    predictions = linear.predict(X_test)
    print('================x_test')
    print(X_test)
    final_models.append(linear)
    real_values.append(y_test)
    pred_values.append(predictions)

print(final_models)
print(real_values)
print(pred_values)

================x_test
    조직변수  공공질서및안전(기재부)     합계출산율  소비자 물가지수  생산자 물가지수  \
11   1.0      0.431567  0.605911  0.466294  0.354854   

    식료품 및 비주류음료품(가계목적별 최종 소비지출)(실질)(원계열)  주류 및 담배(가계목적별 최종 소비지출)(실질)(원계열)  \
11                              0.379744                         0.478867   

    의료보건(가계목적별 최종 소비지출)(실질)(원계열)  국내총생산(실질)  기대여명(0세)(전체)  ...  \
11                      0.444474   0.475398      0.454545  ...   

    영양교육 및 상담 경험률(10~18세)  가공식품 선택 시 영양표시 이용률(6~9세)  \
11               0.807971                  0.480769   

    가공식품 선택 시 영양표시 이용률(10~18세)  영양섭취부족자 분율(1~9세)  영양섭취부족자 분율(10~18세)  \
11                     0.41573          0.189189            0.521368   

    에너지/지방과잉섭취자 분율(1~9세)  에너지/지방과잉섭취자 분율(10~18세)  건강식생활실천율(6~9세)  \
11              0.595238                0.857143        0.355556   

    건강식생활실천율(10~18세)  청소년 비만율(중고등학생)  
11             0.504        0.397351  

[1 rows x 106 columns]
================x_test
    조직변수  공공질서및안전(기재부)     합계출산율  소비자 물가지수  생산자 물가지수  \
12

In [24]:
# 4. RandomForest 모델
param_grid = [
    {'n_estimators': [10, 50, 100]},
    {'max_depth': [None, 10, 20]},
    {'min_samples_split': [2, 5, 10]},
    {'min_samples_leaf': [1, 2, 4]}
]

for X_train, y_train, X_test, y_test in train_test_splits:
    random_forest = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(random_forest, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    print("최적의 하이퍼파라미터:", grid_search.best_params_)
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(X_test)
    final_models.append(best_model)
    real_values.append(y_test)
    pred_values.append(predictions)

print(final_models)
print(real_values)
print(pred_values)


최적의 하이퍼파라미터: {'n_estimators': 100}
최적의 하이퍼파라미터: {'n_estimators': 100}
최적의 하이퍼파라미터: {'n_estimators': 10}
최적의 하이퍼파라미터: {'n_estimators': 50}
최적의 하이퍼파라미터: {'n_estimators': 50}
최적의 하이퍼파라미터: {'n_estimators': 100}
최적의 하이퍼파라미터: {'n_estimators': 100}
최적의 하이퍼파라미터: {'n_estimators': 100}
최적의 하이퍼파라미터: {'n_estimators': 100}
최적의 하이퍼파라미터: {'n_estimators': 100}
최적의 하이퍼파라미터: {'n_estimators': 100}
최적의 하이퍼파라미터: {'n_estimators': 100}
최적의 하이퍼파라미터: {'n_estimators': 100}
[Lasso(alpha=1), Lasso(alpha=5), Lasso(alpha=100), Lasso(alpha=3), Lasso(alpha=3), Lasso(alpha=0.1), Lasso(alpha=1), Lasso(alpha=1), Lasso(alpha=1), Lasso(alpha=1), Lasso(alpha=3), Lasso(alpha=3), Lasso(alpha=3), Ridge(alpha=1), Ridge(alpha=1), Ridge(alpha=1), Ridge(alpha=1), Ridge(alpha=3), Ridge(alpha=3), Ridge(alpha=3), Ridge(alpha=0.0001), Ridge(alpha=3), Ridge(alpha=0.0001), Ridge(alpha=0.0001), Ridge(alpha=0.0001), Ridge(alpha=0.01), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), Line

In [25]:
# 5. GradientBoosting 모델
param_grid = [
    {'n_estimators': [10, 50, 100]},
    {'learning_rate': [0.01, 0.1, 0.5]},
    {'max_depth': [3, 5, 10]},
    {'min_samples_split': [2, 5, 10]},
    {'min_samples_leaf': [1, 2, 4]}
]

for X_train, y_train, X_test, y_test in train_test_splits:
    gradient_boosting = GradientBoostingRegressor(random_state=42)
    grid_search = GridSearchCV(gradient_boosting, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    print("최적의 하이퍼파라미터:", grid_search.best_params_)
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(X_test)
    final_models.append(best_model)
    real_values.append(y_test)
    pred_values.append(predictions)

print(final_models)
print(real_values)
print(pred_values)


최적의 하이퍼파라미터: {'min_samples_split': 5}
최적의 하이퍼파라미터: {'learning_rate': 0.5}
최적의 하이퍼파라미터: {'min_samples_split': 5}
최적의 하이퍼파라미터: {'min_samples_split': 5}
최적의 하이퍼파라미터: {'learning_rate': 0.5}
최적의 하이퍼파라미터: {'max_depth': 5}
최적의 하이퍼파라미터: {'n_estimators': 100}
최적의 하이퍼파라미터: {'min_samples_split': 10}
최적의 하이퍼파라미터: {'min_samples_split': 10}
최적의 하이퍼파라미터: {'min_samples_split': 10}
최적의 하이퍼파라미터: {'learning_rate': 0.5}
최적의 하이퍼파라미터: {'min_samples_split': 5}
최적의 하이퍼파라미터: {'max_depth': 10}
[Lasso(alpha=1), Lasso(alpha=5), Lasso(alpha=100), Lasso(alpha=3), Lasso(alpha=3), Lasso(alpha=0.1), Lasso(alpha=1), Lasso(alpha=1), Lasso(alpha=1), Lasso(alpha=1), Lasso(alpha=3), Lasso(alpha=3), Lasso(alpha=3), Ridge(alpha=1), Ridge(alpha=1), Ridge(alpha=1), Ridge(alpha=1), Ridge(alpha=3), Ridge(alpha=3), Ridge(alpha=3), Ridge(alpha=0.0001), Ridge(alpha=3), Ridge(alpha=0.0001), Ridge(alpha=0.0001), Ridge(alpha=0.0001), Ridge(alpha=0.01), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), Lin

In [26]:
print(file_name)
print(final_models)
print(real_values)
print(pred_values)

식품 아리마
[Lasso(alpha=1), Lasso(alpha=5), Lasso(alpha=100), Lasso(alpha=3), Lasso(alpha=3), Lasso(alpha=0.1), Lasso(alpha=1), Lasso(alpha=1), Lasso(alpha=1), Lasso(alpha=1), Lasso(alpha=3), Lasso(alpha=3), Lasso(alpha=3), Ridge(alpha=1), Ridge(alpha=1), Ridge(alpha=1), Ridge(alpha=1), Ridge(alpha=3), Ridge(alpha=3), Ridge(alpha=3), Ridge(alpha=0.0001), Ridge(alpha=3), Ridge(alpha=0.0001), Ridge(alpha=0.0001), Ridge(alpha=0.0001), Ridge(alpha=0.01), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), RandomForestRegressor(random_state=42), RandomForestRegressor(random_state=42), RandomForestRegressor(n_estimators=10, random_state=42), RandomForestRegressor(n_estimators=50, random_state=42), RandomForestRegressor(n_estimators=50, random_state=42), RandomForestRegressor(random_state=42)

In [27]:
def df_2123(df_results, file_name, final_models, real_values, pred_values):
    start_index = 0

    for model_index in range(len(final_models)):
        for j in range(len(real_values[model_index])):
            df_results.loc[start_index] = [file_name, final_models[model_index], real_values[model_index][j], pred_values[model_index][j]]
            start_index += 1

    return df_results

def df_2123(df_results, file_name, final_models, real_values, pred_values):
    idx = 0
    for i in range(len(final_models)):
        for j in range(len(real_values[i])):
            df_results.loc[idx] = [file_name, final_models[i], real_values[i][j], pred_values[i][j]]
            idx += 1

    return df_results

def df_2021(df_results, file_name, final_models, real_values, pred_values):
    idx = 0

    for i in range(len(final_models)):
        for j in range(len(real_values[i])):
            df_results.loc[idx] = [file_name, final_models[i], real_values[i][j], pred_values[i][j]]
            idx += 1

    return df_results


def df_23n(df_results, file_name, final_models, real_values, pred_values):
    idx = 0

    for i in range(len(final_models)):
        for j in range(len(real_values[i])):
            df_results.loc[idx] = [file_name, final_models[i], real_values[i][j], pred_values[i][j]]
            idx += 1

    return df_results


# 테이블 열 이름 정의
columns = ['file_name', '최종모델', '실제값', '예측값']

# 2021년, 2022년, 2023년 데이터를 위한 빈 데이터프레임 생성
df_results = pd.DataFrame(columns=columns)

# if file_number in [0, 1, 2, 3]:
#     final_dataset = df_2123(df_results, file_name, final_models, real_values, pred_values)
# elif file_number in [4, 5, 6, 7]:
#     final_dataset = df_2021(df_results, file_name, final_models, real_values, pred_values)
# elif file_number == 8:
#     final_dataset = df_23n(df_results, file_name, final_models, real_values, pred_values)

final_dataset = df_23n(df_results, file_name, final_models, real_values, pred_values)
df_results.to_excel(writer, sheet_name=f'결과{file_name}')

In [29]:
#3. 작성 완료 후 파일 저장
writer.close()